In [1]:
!pip install tqdm opencv-python==4.5.2.52 opencv-python-headless==4.5.2.52

ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Could not find a version that satisfies the requirement opencv-python==4.5.2.52 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84, 4.11.0.86)
ERROR: No matching distribution found for opencv-python==4.5.2.52


In [2]:
!pip install remotezip

In [3]:
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 101.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.6 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 65.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.8 MB/

In [4]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

import remotezip as rz
from collections import defaultdict
import pandas as pd


2025-06-05 15:24:10.195899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749137050.217997      35 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749137050.224705      35 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749137050.242786      35 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749137050.242808      35 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749137050.242811      35 computation_placer.cc:177] computation placer alr

In [5]:
# Load the CSV
labels_df = pd.read_csv('/kaggle/input/changing-tires/labels.csv')

action_to_label = {}
for path in labels_df['video_path']:
    folder = path.split('/')[0]
    label = labels_df[labels_df['video_path'].str.startswith(folder)].iloc[0]['label']
    action_to_label[folder] = label

video_counts = defaultdict(int)
for path in labels_df['video_path']:
    folder = path.split('/')[0]
    video_counts[folder] += 1


print("Action to Label Mapping:")
for action, label in action_to_label.items():
    print(f"{action}: {label}")

print("=" * 40)

print("Video Counts per Action:")
for action, count in video_counts.items():
    print(f"{action}: {count} videos")


Action to Label Mapping:
hand_tighten_bolts: 0
initial_wrench_tighten: 1
lift_car_with_jack: 2
loosen_bolts: 3
lower_car: 4
place_spare_tire: 5
remove_bolts: 6
remove_tire: 7
tighten_bolts: 8
Video Counts per Action:
hand_tighten_bolts: 100 videos
initial_wrench_tighten: 100 videos
lift_car_with_jack: 100 videos
loosen_bolts: 100 videos
lower_car: 100 videos
place_spare_tire: 100 videos
remove_bolts: 100 videos
remove_tire: 77 videos
tighten_bolts: 100 videos


In [6]:
import tensorflow as tf
import random
import numpy as np
import cv2
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import os # Added for os.system

# Official TensorFlow Model Garden imports for MoViNet
# Ensure this library is installed: pip install tf-models-official
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

# Frame processing functions
def format_frames(frame, output_size=(224, 224)):
    """
    Pad and resize an image from a video.
    
    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

def frames_from_video_file(video_path, n_frames, output_size=(224, 224), frame_step=15):
    """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.
      frame_step: Number of frames to step over after each selected frame.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
    """
    result = []
    src = cv2.VideoCapture(str(video_path)) 

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        # Ensure max_start is non-negative before passing to randint
        start = random.randint(0, int(max(0, max_start)))


    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    ret, frame = src.read()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result.append(format_frames(frame_rgb, output_size))
    else:
        print(f"Warning: Could not read first frame from {video_path}. Appending zeros.")
        result.append(tf.zeros((*output_size, 3), dtype=tf.float32))

    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_formatted = format_frames(frame_rgb, output_size)
            result.append(frame_formatted)
        else:
            if len(result) > 0:
                 result.append(tf.zeros_like(result[0]))
            else:
                 result.append(tf.zeros((*output_size, 3), dtype=tf.float32))
    src.release()
    
    if not all(isinstance(r, tf.Tensor) for r in result):
        print("Warning: Not all frames are tensors. This might indicate issues in frame extraction.")
        result = [tf.convert_to_tensor(r, dtype=tf.float32) if not isinstance(r, tf.Tensor) else r for r in result]

    if result and len(result) == n_frames:
        try:
            result_array = tf.stack(result, axis=0).numpy()
        except tf.errors.InvalidArgumentError as e:
            print(f"Error stacking frames for {video_path}: {e}. Shape of results: {[r.shape for r in result]}")
            return np.zeros((n_frames, *output_size, 3), dtype=np.float32)
    elif not result:
         print(f"Warning: No frames extracted for {video_path}. Returning zeros.")
         return np.zeros((n_frames, *output_size, 3), dtype=np.float32)
    else: 
         print(f"Warning: Extracted {len(result)} frames, expected {n_frames} for {video_path}. Returning zeros.")
         return np.zeros((n_frames, *output_size, 3), dtype=np.float32)
    return result_array

class FrameGenerator:
    def __init__(self, data_dir, split_df, n_frames, training=False, class_names_list=None):
        self.data_dir = Path(data_dir)
        self.split_df = split_df 
        self.n_frames = n_frames
        self.training = training
        
        self.video_paths = [self.data_dir / path for path in self.split_df['video_path']]
        self.labels = self.split_df['label'].tolist()
        
        if class_names_list:
            self.class_names = class_names_list
        else:
            unique_sorted_labels = sorted(self.split_df['label'].unique())
            self.class_names = [f"class_{i}" for i in unique_sorted_labels]

    def __call__(self):
        pairs = list(zip(self.video_paths, self.labels))
        if self.training:
            random.shuffle(pairs)
        
        for path, label in pairs:
            # Pass resolution to frames_from_video_file
            video_frames = frames_from_video_file(path, self.n_frames, (resolution, resolution))
            yield video_frames, tf.cast(label, tf.int16)

# --- Main script starts ---
data_dir = Path("/kaggle/input/changing-tires") 
n_frames = 10 
resolution = 224 

try:
    df = pd.read_csv(data_dir / "labels.csv")
except FileNotFoundError:
    print(f"Error: labels.csv not found at {data_dir / 'labels.csv'}")
    df = pd.DataFrame({
        'video_path': [f'dummy_class/video{i}.mp4' for i in range(10)],
        'label': [i % 2 for i in range(10)] 
    })
    # Create dummy video files if using dummy data for testing locally
    # For kaggle, this part might not be needed if using actual data
    if str(data_dir) == ".": # If using local dummy data
        for i in range(10):
            dummy_video_path = Path(f"dummy_class/video{i}.mp4")
            dummy_video_path.parent.mkdir(parents=True, exist_ok=True)
            # Create a very short, tiny dummy mp4 if opencv can handle it
            # Otherwise, this part needs actual dummy video files.
            # For now, just creating empty files for path existence.
            if not dummy_video_path.exists():
                 # Creating an empty file, opencv will fail to read but path exists
                with open(dummy_video_path, 'w') as fp:
                    pass # create empty file
        print("Created dummy video file paths for local testing.")


num_classes = df['label'].nunique()
print(f"Number of classes in the dataset: {num_classes}")

unique_sorted_labels = sorted(df['label'].unique())
class_names_list = [f"action_{label_val}" for label_val in unique_sorted_labels]


train_df, temp_df = train_test_split(
    df, 
    test_size=0.3, 
    stratify=df['label'] if num_classes > 1 and len(df) >= num_classes * 2 else None,
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5, 
    stratify=temp_df['label'] if num_classes > 1 and len(temp_df) >= num_classes * 2 else None,
    random_state=42
)

# Corrected percentage calculation and formatting
train_percentage = (len(train_df) / len(df) * 100) if len(df) > 0 else 0.0
val_percentage = (len(val_df) / len(df) * 100) if len(df) > 0 else 0.0
test_percentage = (len(test_df) / len(df) * 100) if len(df) > 0 else 0.0

print(f"Train set: {len(train_df)} samples ({train_percentage:.1f}%)")
print(f"Validation set: {len(val_df)} samples ({val_percentage:.1f}%)")
print(f"Test set: {len(test_df)} samples ({test_percentage:.1f}%)")

Number of classes in the dataset: 9
Train set: 613 samples (69.9%)
Validation set: 132 samples (15.1%)
Test set: 132 samples (15.1%)


In [7]:
train_generator = FrameGenerator(data_dir, train_df, n_frames, training=True, class_names_list=class_names_list)
val_generator = FrameGenerator(data_dir, val_df, n_frames, training=False, class_names_list=class_names_list)
test_generator = FrameGenerator(data_dir, test_df, n_frames, training=False, class_names_list=class_names_list)

output_signature = (
    tf.TensorSpec(shape=(n_frames, resolution, resolution, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

train_ds = tf.data.Dataset.from_generator(train_generator, output_signature=output_signature).cache().prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_generator(val_generator, output_signature=output_signature).cache().prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_generator(test_generator, output_signature=output_signature).cache().prefetch(tf.data.AUTOTUNE)

batch_size = 8 
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

I0000 00:00:1749137059.449912      35 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [8]:
model_id = 'a0'
tf.keras.backend.clear_session()

checkpoint_dir_name = f'movinet_{model_id}_base'
checkpoint_dir_path = Path(checkpoint_dir_name)

if not checkpoint_dir_path.exists():
    print(f"Downloading and extracting {model_id} weights...")
    wget_command = f"wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_{model_id}_base.tar.gz -O movinet_{model_id}_base.tar.gz -q"
    tar_command = f"tar -xvf movinet_{model_id}_base.tar.gz"
    if os.system(wget_command) == 0:
        os.system(tar_command)
    else:
        print(f"Failed to download weights. Ensure wget and tar are installed or download manually.")
        if not checkpoint_dir_path.exists(): print(f"Checkpoint dir {checkpoint_dir_path} still not found.")
else:
    print(f"Checkpoint directory {checkpoint_dir_path} already exists.")

checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir_path)
if not checkpoint_path: print(f"Error: No checkpoint in {checkpoint_dir_path}.")
else: print(f"Found checkpoint: {checkpoint_path}")

movinet_a0_base/
movinet_a0_base/checkpoint
movinet_a0_base/ckpt-1.data-00000-of-00001
movinet_a0_base/ckpt-1.index
Found checkpoint: movinet_a0_base/ckpt-1


In [9]:
# 1. Create a temporary non-causal model to load the full checkpoint
temp_backbone_non_causal = movinet.Movinet(model_id=model_id, causal=False)
temp_model_for_loading = movinet_model.MovinetClassifier(backbone=temp_backbone_non_causal, num_classes=600)
dummy_input_shape_for_loading = [1, n_frames, resolution, resolution, 3]
temp_model_for_loading.build(dummy_input_shape_for_loading)

if checkpoint_path:
    print("Loading pre-trained weights into temporary non-causal model...")
    load_checkpoint = tf.train.Checkpoint(model=temp_model_for_loading)
    status = load_checkpoint.restore(checkpoint_path)
    try:
        status.assert_existing_objects_matched()
        print("Successfully loaded pre-trained weights into temporary model.")
    except AssertionError as e:
        print(f"Warning: Weight loading status: {e}. Some weights might not have matched.")
        status.expect_partial()
else:
    print("Skipping weight loading as checkpoint path is not available.")

# 2. Create the backbone for training (causal, but NOT using external states for fit())
training_backbone = movinet.Movinet(
    model_id=model_id,
    causal=True,
    use_external_states=False # Important for training with model.fit()
)

# 3. Create the model for training
# output_states=False because standard loss functions don't expect states
model_for_training = movinet_model.MovinetClassifier(
    backbone=training_backbone,
    num_classes=num_classes, # Your dataset's number of classes
    output_states=False      # Important for training with model.fit()
)

# 4. Build the training model
model_for_training.build([None, n_frames, resolution, resolution, 3])

# 5. Transfer backbone weights from the loaded non-causal backbone to the training backbone
if checkpoint_path and hasattr(temp_model_for_loading, 'backbone') and hasattr(model_for_training, 'backbone'):
    print("Transferring backbone weights from temporary model to training model...")
    try:
        model_for_training.backbone.set_weights(temp_model_for_loading.backbone.get_weights())
        print("Successfully transferred backbone weights to training model.")
    except ValueError as e:
        print(f"Error transferring backbone weights to training model: {e}")
else:
     print("Skipping backbone weight transfer to training model.")

# 6. Set backbone trainability
model_for_training.backbone.trainable = True 

# --- Fine-tuning ---
num_epochs = 4
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model_for_training.compile(
    loss=loss_obj,
    optimizer='adam',
    metrics=['accuracy']
)


Loading pre-trained weights into temporary non-causal model...
Successfully loaded pre-trained weights into temporary model.
Transferring backbone weights from temporary model to training model...
Successfully transferred backbone weights to training model.


In [10]:
history = model_for_training.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    verbose=1,
)

Epoch 1/5


I0000 00:00:1749137134.842644     126 service.cc:152] XLA service 0x78e814361c40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749137134.842681     126 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1749137134.870227     126 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749137135.795285     126 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-06-05 15:25:37.641511: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


77/77 [==============================] - 720s 8s/step - loss: 1.3019 - accuracy: 0.5334 - val_loss: 1.2904 - val_accuracy: 0.5530
Epoch 2/5
77/77 [==============================] - 22s 282ms/step - loss: 0.3840 - accuracy: 0.8695 - val_loss: 1.2906 - val_accuracy: 0.6818
Epoch 3/5
77/77 [==============================] - 22s 280ms/step - loss: 0.1596 - accuracy: 0.9543 - val_loss: 1.0502 - val_accuracy: 0.7879
Epoch 4/5
77/77 [==============================] - 21s 279ms/step - loss: 0.1445 - accuracy: 0.9511 - val_loss: 0.5813 - val_accuracy: 0.8333
Epoch 5/5
77/77 [==============================] - 22s 279ms/step - loss: 0.1497 - accuracy: 0.9560 - val_loss: 0.4868 - val_accuracy: 0.8636


In [11]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
plt.savefig("training_history_plots.png") # Save the plot


In [12]:
print("Evaluating on the test set with the training model...")
loss, accuracy = model_for_training.evaluate(test_ds, verbose=1)
print(f"Test Loss (training model): {loss:.4f}")
print(f"Test Accuracy (training model): {accuracy:.4f}")



Evaluating on the test set with the training model...
17/17 [==============================] - 91s 5s/step - loss: 0.2863 - accuracy: 0.9318
Test Loss (training model): 0.2863
Test Accuracy (training model): 0.9318


In [13]:
def get_actual_predicted_labels(dataset):
  """
    Create a list of actual ground truth values and the predictions from the model.

    Args:
      dataset: An iterable data structure, such as a TensorFlow Dataset, with features and labels.

    Return:
      Ground truth and predicted values for a particular dataset.
  """
  actual = [labels for _, labels in dataset.unbatch()]
  predicted = model_for_training.predict(dataset)

  actual = tf.stack(actual, axis=0)
  predicted = tf.concat(predicted, axis=0)
  predicted = tf.argmax(predicted, axis=1)

  return actual, predicted

def plot_confusion_matrix(actual, predicted, labels, ds_type, filename="confusion_matrix.png"):
  
  cm = tf.math.confusion_matrix(actual, predicted)
  plt.figure(figsize=(12, 12))
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(font_scale=1.4) 
  ax.set_title('Confusion matrix of action recognition for ' + ds_type)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

  plt.savefig(filename, bbox_inches='tight')
  print(f"Confusion matrix saved to {filename}")
  plt.show()

In [14]:
label_names = list(train_generator.class_names)
actual, predicted = get_actual_predicted_labels(test_ds)
plot_confusion_matrix(actual, predicted, label_names, 'test')

17/17 [==============================] - 32s 800ms/step
Confusion matrix saved to confusion_matrix.png


In [15]:
from sklearn.metrics import classification_report


print(classification_report(actual, predicted, target_names=label_names))


              precision    recall  f1-score   support

    action_0       0.79      1.00      0.88        15
    action_1       1.00      0.67      0.80        15
    action_2       1.00      0.87      0.93        15
    action_3       0.94      1.00      0.97        15
    action_4       0.88      1.00      0.94        15
    action_5       0.88      1.00      0.94        15
    action_6       1.00      0.93      0.97        15
    action_7       1.00      1.00      1.00        12
    action_8       1.00      0.93      0.97        15

    accuracy                           0.93       132
   macro avg       0.94      0.93      0.93       132
weighted avg       0.94      0.93      0.93       132



In [16]:

# --- Streaming Inference Example ---
print("\nSetting up model for streaming inference...")
# 1. Create backbone for streaming (causal=True, use_external_states=True)
streaming_backbone = movinet.Movinet(
    model_id=model_id,
    causal=True,
    use_external_states=True # Crucial for streaming
)
# 2. Create classifier for streaming (output_states=True)
streaming_model = movinet_model.MovinetClassifier(
    backbone=streaming_backbone,
    num_classes=num_classes,
    output_states=True # Crucial for getting states back
)

# 3. Build the streaming model by calling it once
# This ensures layers are built and input/state shapes are understood.
streaming_batch_size_build = 1
dummy_frame_shape_for_build = [streaming_batch_size_build, 1, resolution, resolution, 3] # B, T=1, H, W, C
dummy_input_frame = tf.zeros(dummy_frame_shape_for_build, dtype=tf.float32)
try:
    # Initialize states for the specific input shape for the streaming model
    initial_build_states = streaming_model.init_states(tf.TensorShape(dummy_frame_shape_for_build))
    print("Building streaming model by performing a dummy call...")
    _ = streaming_model((dummy_input_frame, initial_build_states), training=False) # Call to build
    print("Streaming model built successfully.")
except Exception as e:
    print(f"Error building streaming model with a dummy call: {e}")
    print("Attempting to build with model.build() as fallback (might be less robust for tuple inputs).")
    # Fallback build (less ideal for models expecting tuple inputs like (frames, states))
    # For MovinetClassifier, the primary input is frames, states are handled internally if use_external_states=False
    # but for use_external_states=True, the call signature expects (frames, states).
    # The dummy call above is generally better. If it fails, there might be deeper issues.
    # This build command might not correctly infer the tuple input structure.
    # streaming_model.build(input_shape=[dummy_frame_shape_for_build, type(initial_build_states)]) # This is complex
    # For now, if dummy call fails, we might not be able to proceed with set_weights reliably.

# 4. Transfer weights from the fine-tuned training model to the streaming model
print("Transferring all weights from fine-tuned training model to streaming model...")
try:
    streaming_model.set_weights(model_for_training.get_weights())
    print("Successfully transferred weights to streaming model.")
except Exception as e:
    print(f"Error transferring weights to streaming model: {e}")
    print("Ensure streaming model was built correctly before setting weights.")


if not test_df.empty:
    example_video_path_str = test_df['video_path'].iloc[0]
    example_video_path = data_dir / example_video_path_str
    
    print(f"\nRunning streaming inference demo on: {example_video_path}")

    streaming_batch_size_demo = 1 
    single_frame_input_shape = tf.TensorShape([streaming_batch_size_demo, 1, resolution, resolution, 3])
    
    try:
        states = streaming_model.init_states(single_frame_input_shape)
    except Exception as e:
        print(f"Error initializing states for streaming demo model: {e}")
        states = None # Cannot proceed with demo if states cannot be initialized

    if states is not None:
        cap = cv2.VideoCapture(str(example_video_path))
        if not cap.isOpened():
            print(f"Error: Could not open video file {example_video_path} for streaming demo.")
        else:
            num_demo_frames = n_frames * 2 
            for i in range(num_demo_frames):
                ret, frame_bgr = cap.read()
                if not ret:
                    print("End of video or error reading frame during streaming demo.")
                    break
                
                frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
                formatted_frame_tensor = format_frames(frame_rgb, (resolution, resolution))
                input_frame_stream = formatted_frame_tensor[tf.newaxis, tf.newaxis, ...] 

                try:
                    logits, states = streaming_model((input_frame_stream, states), training=False)
                    predicted_class_id = tf.argmax(logits, axis=-1).numpy()[0]
                    predicted_label_name = class_names_list[predicted_class_id] if predicted_class_id < len(class_names_list) else f"unknown_id_{predicted_class_id}"
                    print(f"Stream Frame {i+1}: Predicted Class ID: {predicted_class_id}, Label: {predicted_label_name}")
                except Exception as e:
                    print(f"Error during streaming inference on frame {i+1}: {e}")
                    break 
            cap.release()
else:
    print("Test set is empty. Skipping streaming inference demo.")

# To save the fine-tuned streaming model (which is `streaming_model`):
# print("Saving the fine-tuned streaming model...")
# streaming_model.save('my_streaming_movinet_finetuned_model') # Saves in SavedModel format
# Note: When loading a SavedModel with custom objects (like MoViNet layers),
# you might need to provide them via custom_objects argument or ensure they are registered.


Setting up model for streaming inference...
Building streaming model by performing a dummy call...
Streaming model built successfully.
Transferring all weights from fine-tuned training model to streaming model...
Successfully transferred weights to streaming model.

Running streaming inference demo on: /kaggle/input/changing-tires/remove_tire/remove_tire_04_piecewise_affine.mp4
Stream Frame 1: Predicted Class ID: 4, Label: action_4
Stream Frame 2: Predicted Class ID: 5, Label: action_5
Stream Frame 3: Predicted Class ID: 7, Label: action_7
Stream Frame 4: Predicted Class ID: 5, Label: action_5
Stream Frame 5: Predicted Class ID: 7, Label: action_7
Stream Frame 6: Predicted Class ID: 7, Label: action_7
Stream Frame 7: Predicted Class ID: 7, Label: action_7
Stream Frame 8: Predicted Class ID: 7, Label: action_7
Stream Frame 9: Predicted Class ID: 6, Label: action_6
Stream Frame 10: Predicted Class ID: 6, Label: action_6
Stream Frame 11: Predicted Class ID: 6, Label: action_6
Stream Fram

In [17]:
list(action_to_label.keys())

['hand_tighten_bolts',
 'initial_wrench_tighten',
 'lift_car_with_jack',
 'loosen_bolts',
 'lower_car',
 'place_spare_tire',
 'remove_bolts',
 'remove_tire',
 'tighten_bolts']

In [18]:
# ========== SAVING THE STREAMING MODEL (Weights Method) ==========
print("Saving the fine-tuned streaming model weights...")

# Save model weights
streaming_model.save_weights('streaming_movinet_weights.h5')
print("✅ Streaming model weights saved!")

# Create and save model configuration 
# Replace these with your actual values if different
import json
config = {
    'model_id': 'a0',  
    'num_classes': 9,  # Change this to your number of classes
    'resolution': 224,  # Change this to your resolution (e.g., 224, 172, etc.)
    'class_names': list(action_to_label.keys()),  # Replace with your actual class names
    'streaming_batch_size': 1
}

# Print current values so you can verify them
print(f"📋 Saving configuration:")
print(f"   Model ID: {config['model_id']}")
print(f"   Num classes: {config['num_classes']}")
print(f"   Resolution: {config['resolution']}")
print(f"   Class names: {config['class_names']}")

with open('/kaggle/working/streaming_model_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print("✅ Model configuration saved!")

# ========== LOADING AND RECONSTRUCTING THE MODEL ==========
import tensorflow as tf
import cv2
import numpy as np
import json
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

# Load configuration
with open('/kaggle/working/streaming_model_config.json', 'r') as f:
    config = json.load(f)

model_id = config['model_id']
num_classes = config['num_classes']
resolution = config['resolution']
class_names_list = config['class_names']
streaming_batch_size = config['streaming_batch_size']

print("Reconstructing streaming model architecture...")

# 1. Create streaming backbone (same as original)
streaming_backbone = movinet.Movinet(
    model_id=model_id,
    causal=True,
    use_external_states=True
)

# 2. Create streaming classifier (same as original)
loaded_streaming_model = movinet_model.MovinetClassifier(
    backbone=streaming_backbone,
    num_classes=num_classes,
    output_states=True
)

# 3. Build the model by calling it once (same as original)
dummy_frame_shape = [streaming_batch_size, 1, resolution, resolution, 3]
dummy_input_frame = tf.zeros(dummy_frame_shape, dtype=tf.float32)

try:
    initial_states = loaded_streaming_model.init_states(tf.TensorShape(dummy_frame_shape))
    print("Building streaming model by performing a dummy call...")
    _ = loaded_streaming_model((dummy_input_frame, initial_states), training=False)
    print("Streaming model built successfully.")
except Exception as e:
    print(f"Error building streaming model: {e}")
    raise e

# 4. Load the saved weights
print("Loading saved weights...")
loaded_streaming_model.load_weights('/kaggle/working/streaming_movinet_weights.h5')
print("✅ Weights loaded successfully!")



Saving the fine-tuned streaming model weights...
✅ Streaming model weights saved!
📋 Saving configuration:
   Model ID: a0
   Num classes: 9
   Resolution: 224
   Class names: ['hand_tighten_bolts', 'initial_wrench_tighten', 'lift_car_with_jack', 'loosen_bolts', 'lower_car', 'place_spare_tire', 'remove_bolts', 'remove_tire', 'tighten_bolts']
✅ Model configuration saved!
Reconstructing streaming model architecture...
Building streaming model by performing a dummy call...
Streaming model built successfully.
Loading saved weights...
✅ Weights loaded successfully!


In [19]:
print(loaded_streaming_model.summary())

Model: "movinet_classifier_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image (InputLayer)          [(None, None, None, None,    0         []                            
                             3)]                                                                  
                                                                                                  
 state_block0_layer0_pool_b  [(None, 1, 1, 1, 24)]        0         []                            
 uffer (InputLayer)                                                                               
                                                                                                  
 state_block0_layer0_pool_f  [(None,)]                    0         []                            
 rame_count (InputLayer)                                                       

In [20]:
# ========== FORMAT FRAMES FUNCTION ==========
def format_frames(frame, output_size):
    """Format frame for model input (same as your original)"""
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

# ========== VIDEO PREDICTION FUNCTION ==========
def predict_video_streaming(model, video_path, class_names, resolution=224, max_frames=None):
    """Streaming video prediction (inspired by your original demo)"""
    print(f"\nRunning streaming inference on: {video_path}")
    
    # Initialize states for streaming
    single_frame_input_shape = tf.TensorShape([1, 1, resolution, resolution, 3])
    
    try:
        states = model.init_states(single_frame_input_shape)
        print("States initialized successfully.")
    except Exception as e:
        print(f"Error initializing states: {e}")
        return []
    
    # Open video
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return []
    
    predictions = []
    frame_count = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    process_frames = min(total_frames, max_frames) if max_frames else total_frames
    
    print(f"Processing {process_frames} frames...")
    
    while frame_count < process_frames:
        ret, frame_bgr = cap.read()
        if not ret:
            print("End of video or error reading frame.")
            break
        
        # Convert BGR to RGB (same as original)
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        
        # Format frame (same as original)
        formatted_frame_tensor = format_frames(frame_rgb, (resolution, resolution))
        input_frame_stream = formatted_frame_tensor[tf.newaxis, tf.newaxis, ...]
        
        try:
            # Streaming inference (same as original)
            logits, states = model((input_frame_stream, states), training=False)
            predicted_class_id = tf.argmax(logits, axis=-1).numpy()[0]
            predicted_label_name = class_names[predicted_class_id] if predicted_class_id < len(class_names) else f"unknown_id_{predicted_class_id}"
            
            # Get confidence
            confidence = tf.nn.softmax(logits).numpy()[0][predicted_class_id]
            
            prediction_info = {
                'frame': frame_count + 1,
                'class_id': int(predicted_class_id),
                'class_name': predicted_label_name,
                'confidence': float(confidence)
            }
            
            predictions.append(prediction_info)
            
            # Print progress (same format as original)
            print(f"Stream Frame {frame_count + 1}: Predicted Class ID: {predicted_class_id}, Label: {predicted_label_name}, Confidence: {confidence:.3f}")
            
        except Exception as e:
            print(f"Error during streaming inference on frame {frame_count + 1}: {e}")
            break
        
        frame_count += 1
    
    cap.release()
    print(f"\nStreaming inference completed! Processed {len(predictions)} frames")
    return predictions

# ========== USAGE EXAMPLE ==========
# Replace with your actual video path
video_path = "/kaggle/input/changing-tires/hand_tighten_bolts/hand_tighten_bolts_13.mp4"

# Check if video exists
from pathlib import Path
if Path(video_path).exists():
    # Run streaming prediction
    results = predict_video_streaming(
        model=loaded_streaming_model,
        video_path=video_path,
        class_names=class_names_list,
        resolution=resolution,
        max_frames=20  # Process first 50 frames, set to None for all frames
    )
    
    # Show summary
    if results:
        print(f"\n📊 PREDICTION SUMMARY:")
        print(f"Total frames: {len(results)}")
        
        # Most common prediction
        from collections import Counter
        class_counts = Counter([r['class_name'] for r in results])
        most_common = class_counts.most_common(1)[0]
        print(f"Most predicted class: {most_common[0]} ({most_common[1]} frames)")
        
        # Average confidence
        avg_confidence = sum([r['confidence'] for r in results]) / len(results)
        print(f"Average confidence: {avg_confidence:.3f}")
        
        
else:
    print(f"❌ Video file not found: {video_path}")
    print("Please update the video_path variable with your actual video file path")


Running streaming inference on: /kaggle/input/changing-tires/hand_tighten_bolts/hand_tighten_bolts_13.mp4
States initialized successfully.
Processing 20 frames...
Stream Frame 1: Predicted Class ID: 4, Label: lower_car, Confidence: 0.182
Stream Frame 2: Predicted Class ID: 4, Label: lower_car, Confidence: 0.196
Stream Frame 3: Predicted Class ID: 5, Label: place_spare_tire, Confidence: 0.325
Stream Frame 4: Predicted Class ID: 5, Label: place_spare_tire, Confidence: 0.563
Stream Frame 5: Predicted Class ID: 5, Label: place_spare_tire, Confidence: 0.653
Stream Frame 6: Predicted Class ID: 5, Label: place_spare_tire, Confidence: 0.510
Stream Frame 7: Predicted Class ID: 0, Label: hand_tighten_bolts, Confidence: 0.568
Stream Frame 8: Predicted Class ID: 0, Label: hand_tighten_bolts, Confidence: 0.623
Stream Frame 9: Predicted Class ID: 0, Label: hand_tighten_bolts, Confidence: 0.683
Stream Frame 10: Predicted Class ID: 0, Label: hand_tighten_bolts, Confidence: 0.733
Stream Frame 11: Pred

In [23]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation, PillowWriter
import tensorflow as tf
from pathlib import Path
from collections import Counter, deque
import json

def format_frames(frame, output_size):
    """Format frame for model input"""
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame

def create_prediction_video_with_plot(model, video_path, class_names, resolution=224, 
                                    max_frames=None, output_path='/kaggle/working/prediction_video.mp4'):
    """
    Create a video showing original frames + real-time prediction plot
    """
    print(f"\nCreating prediction visualization for: {video_path}")
    
    # Initialize states for streaming
    single_frame_input_shape = tf.TensorShape([1, 1, resolution, resolution, 3])
    
    try:
        states = model.init_states(single_frame_input_shape)
        print("States initialized successfully.")
    except Exception as e:
        print(f"Error initializing states: {e}")
        return None
    
    # Open input video
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    process_frames = min(total_frames, max_frames) if max_frames else total_frames
    print(f"Processing {process_frames} frames at {fps} FPS...")
    
    # Prepare data storage
    predictions = []
    confidence_history = []
    class_history = []
    frames_data = []
    
    # Colors for each class
    colors = plt.cm.Set3(np.linspace(0, 1, len(class_names)))
    class_colors = {class_name: colors[i] for i, class_name in enumerate(class_names)}
    
    frame_count = 0
    
    # Process all frames first
    print("Processing frames for predictions...")
    while frame_count < process_frames:
        ret, frame_bgr = cap.read()
        if not ret:
            break
        
        # Store original frame
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        frames_data.append(frame_rgb.copy())
        
        # Format frame for prediction
        formatted_frame_tensor = format_frames(frame_rgb, (resolution, resolution))
        input_frame_stream = formatted_frame_tensor[tf.newaxis, tf.newaxis, ...]
        
        try:
            # Streaming inference
            logits, states = model((input_frame_stream, states), training=False)
            predicted_class_id = tf.argmax(logits, axis=-1).numpy()[0]
            predicted_label_name = class_names[predicted_class_id]
            
            # Get all class confidences
            all_confidences = tf.nn.softmax(logits).numpy()[0]
            max_confidence = all_confidences[predicted_class_id]
            
            prediction_info = {
                'frame': frame_count + 1,
                'class_id': int(predicted_class_id),
                'class_name': predicted_label_name,
                'confidence': float(max_confidence),
                'all_confidences': all_confidences.tolist()
            }
            
            predictions.append(prediction_info)
            confidence_history.append(max_confidence)
            class_history.append(predicted_label_name)
            
            if (frame_count + 1) % 10 == 0:
                print(f"Processed frame {frame_count + 1}/{process_frames}")
            
        except Exception as e:
            print(f"Error during inference on frame {frame_count + 1}: {e}")
            break
        
        frame_count += 1
    
    cap.release()
    
    if not predictions:
        print("No predictions generated!")
        return None
    
    print(f"Generating visualization video with {len(predictions)} frames...")
    
    # Setup video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (1800, 600))  # Width, Height matching figsize
    
    # Window for moving average
    window_size = min(10, len(predictions))
    
    for i in range(len(predictions)):
        # Create new figure for each frame - this fixes the figsize issue
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
        plt.tight_layout(pad=3.0)
        
        # Current prediction data
        current_pred = predictions[i]
        current_frame = frames_data[i]
        
        # 1. Show original frame with prediction overlay
        ax1.imshow(current_frame)
        ax1.set_title(f'Frame {i+1}/{len(predictions)}\nPrediction: {current_pred["class_name"]}\nConfidence: {current_pred["confidence"]:.3f}', 
                     fontsize=12, fontweight='bold')
        ax1.axis('off')
        
        # Add prediction text overlay
        bbox_props = dict(boxstyle="round,pad=0.3", facecolor=class_colors[current_pred["class_name"]], alpha=0.8)
        ax1.text(0.02, 0.98, f'{current_pred["class_name"]}\n{current_pred["confidence"]:.3f}', 
                transform=ax1.transAxes, fontsize=14, fontweight='bold',
                verticalalignment='top', bbox=bbox_props, color='white')
        
        # 2. Confidence over time
        frames_so_far = list(range(1, i+2))
        confidences_so_far = confidence_history[:i+1]
        
        ax2.plot(frames_so_far, confidences_so_far, 'b-', linewidth=2, alpha=0.7)
        ax2.scatter(frames_so_far[-1], confidences_so_far[-1], color='red', s=100, zorder=5)
        
        # Moving average
        if len(confidences_so_far) >= window_size:
            moving_avg = []
            for j in range(window_size-1, len(confidences_so_far)):
                avg = np.mean(confidences_so_far[j-window_size+1:j+1])
                moving_avg.append(avg)
            
            avg_frames = frames_so_far[window_size-1:]
            ax2.plot(avg_frames, moving_avg, 'r--', linewidth=2, alpha=0.8, label=f'Moving Avg ({window_size})')
            ax2.legend()
        
        ax2.set_xlabel('Frame Number')
        ax2.set_ylabel('Confidence')
        ax2.set_title('Prediction Confidence Over Time')
        ax2.grid(True, alpha=0.3)
        ax2.set_ylim(0, 1)
        ax2.set_xlim(1, len(predictions))
        
        # 3. Class distribution and current class probabilities
        # Top: Class distribution over time
        ax3_top = ax3
        class_counts = Counter(class_history[:i+1])
        classes = list(class_counts.keys())
        counts = list(class_counts.values())
        
        bars = ax3_top.bar(classes, counts, color=[class_colors[cls] for cls in classes], alpha=0.7)
        ax3_top.set_xlabel('Classes')
        ax3_top.set_ylabel('Frame Count')
        ax3_top.set_title('Class Distribution Over Time')
        ax3_top.tick_params(axis='x', rotation=45)
        
        # Highlight current prediction
        for j, cls in enumerate(classes):
            if cls == current_pred["class_name"]:
                bars[j].set_alpha(1.0)
                bars[j].set_edgecolor('black')
                bars[j].set_linewidth(2)
        
        # Add count labels on bars
        for bar, count in zip(bars, counts):
            height = bar.get_height()
            ax3_top.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                        f'{count}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        
        # Convert matplotlib figure to opencv image
        fig.canvas.draw()
        img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        img = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # Resize to match video writer dimensions if needed
        if img_bgr.shape[:2] != (600, 1800):
            img_bgr = cv2.resize(img_bgr, (1800, 600))
        
        out.write(img_bgr)
        
        # Close the figure to free memory
        plt.close(fig)
        
        if (i + 1) % 20 == 0:
            print(f"Generated frame {i + 1}/{len(predictions)} of visualization")
    
    out.release()
    
    print(f"✅ Visualization video saved to: {output_path}")
    
    # Save detailed results
    results_data = {
        'video_info': {
            'original_path': str(video_path),
            'total_frames_processed': len(predictions),
            'fps': fps,
            'resolution': resolution
        },
        'predictions': predictions,
        'summary_stats': {
            'class_distribution': dict(Counter(class_history)),
            'average_confidence': float(np.mean(confidence_history)),
            'confidence_std': float(np.std(confidence_history)),
            'most_confident_frame': int(np.argmax(confidence_history) + 1),
            'max_confidence': float(np.max(confidence_history))
        }
    }
    
    with open('/kaggle/working/detailed_predictions.json', 'w') as f:
        json.dump(results_data, f, indent=2)
    
    return results_data

# ========== USAGE EXAMPLE ==========
video_path = "/kaggle/input/changing-tires/hand_tighten_bolts/hand_tighten_bolts_13.mp4"

# Check if video exists
if Path(video_path).exists():
    print("🎬 Creating prediction visualization...")
    
    # Create visualization video
    results = create_prediction_video_with_plot(
        model=loaded_streaming_model,
        video_path=video_path,
        class_names=class_names_list,
        resolution=resolution,
        max_frames=50,  # Adjust based on your needs
        output_path='/kaggle/working/prediction_visualization.mp4'
    )
    
    if results:
        print(f"\n📊 FINAL SUMMARY:")
        print(f"Total frames processed: {results['video_info']['total_frames_processed']}")
        print(f"Average confidence: {results['summary_stats']['average_confidence']:.3f}")
        print(f"Most confident prediction: Frame {results['summary_stats']['most_confident_frame']} ({results['summary_stats']['max_confidence']:.3f})")
        
        print(f"\n📈 Class Distribution:")
        for class_name, count in results['summary_stats']['class_distribution'].items():
            percentage = (count / results['video_info']['total_frames_processed']) * 100
            print(f"  {class_name}: {count} frames ({percentage:.1f}%)")
        
        print(f"\n💾 Files saved:")
        print(f"  - Visualization video: /kaggle/working/prediction_visualization.mp4")
        print(f"  - Detailed results: /kaggle/working/detailed_predictions.json")
        
else:
    print(f"❌ Video file not found: {video_path}")

# ========== SIMPLE PLOT GENERATION (Alternative) ==========
def create_simple_confidence_plot(predictions, output_path='/kaggle/working/confidence_plot.png'):
    """Create a simple static plot of confidence over time"""
    
    frames = [p['frame'] for p in predictions]
    confidences = [p['confidence'] for p in predictions]
    class_names_frames = [p['class_name'] for p in predictions]
    
    plt.figure(figsize=(12, 8))
    
    # Main confidence plot
    plt.subplot(2, 1, 1)
    plt.plot(frames, confidences, 'b-', linewidth=2, alpha=0.7)
    plt.scatter(frames, confidences, c=[hash(name) for name in class_names_frames], 
                cmap='tab10', s=50, alpha=0.8)
    
    # Moving average
    window = 5
    if len(confidences) >= window:
        moving_avg = []
        for i in range(window-1, len(confidences)):
            avg = np.mean(confidences[i-window+1:i+1])
            moving_avg.append(avg)
        plt.plot(frames[window-1:], moving_avg, 'r--', linewidth=2, label=f'Moving Average ({window})')
        plt.legend()
    
    plt.xlabel('Frame Number')
    plt.ylabel('Confidence')
    plt.title('Prediction Confidence Over Time')
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1)
    
    # Class distribution
    plt.subplot(2, 1, 2)
    class_counts = Counter(class_names_frames)
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    
    bars = plt.bar(classes, counts, alpha=0.7)
    plt.xlabel('Classes')
    plt.ylabel('Frame Count')
    plt.title('Class Distribution')
    plt.xticks(rotation=45)
    
    # Add count labels
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{count}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"📊 Confidence plot saved to: {output_path}")

# Uncomment to create simple static plot instead of video
# if 'results' in locals() and results:
#     create_simple_confidence_plot(results['predictions'])

🎬 Creating prediction visualization...

Creating prediction visualization for: /kaggle/input/changing-tires/hand_tighten_bolts/hand_tighten_bolts_13.mp4
States initialized successfully.
Processing 50 frames at 30 FPS...
Processing frames for predictions...
Processed frame 10/50
Processed frame 20/50
Processed frame 30/50
Processed frame 40/50
Processed frame 50/50
Generating visualization video with 50 frames...


/tmp/ipykernel_35/20757013.py:194: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Generated frame 20/50 of visualization
Generated frame 40/50 of visualization
✅ Visualization video saved to: /kaggle/working/prediction_visualization.mp4

📊 FINAL SUMMARY:
Total frames processed: 50
Average confidence: 0.802
Most confident prediction: Frame 45 (0.989)

📈 Class Distribution:
  lower_car: 2 frames (4.0%)
  place_spare_tire: 4 frames (8.0%)
  hand_tighten_bolts: 44 frames (88.0%)

💾 Files saved:
  - Visualization video: /kaggle/working/prediction_visualization.mp4
  - Detailed results: /kaggle/working/detailed_predictions.json
